In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

Read the data using the Kaggle Overview as a guide for parsing. Then save into a dataframe

In [2]:
# Read the JSON file
with open('games.json', 'r', encoding='utf-8') as fin:
    dataset = json.load(fin)

# Create a list to store the flattened data
games_data = []

for app_id, game in dataset.items():
    # Flatten nested lists into comma-separated strings
    developers_str = ','.join(game['developers']) if game['developers'] else ''
    publishers_str = ','.join(game['publishers']) if game['publishers'] else ''
    categories_str = ','.join(game['categories']) if game['categories'] else ''
    genres_str = ','.join(game['genres']) if game['genres'] else ''
    tags_str = ','.join(str(tag) for tag in game['tags']) if game['tags'] else ''
    screenshots_str = ','.join(game['screenshots']) if game['screenshots'] else ''
    movies_str = ','.join(game['movies']) if game['movies'] else ''

    # Create a flattened dictionary for this game
    game_dict = {
        'AppID': app_id,
        'Name': game['name'],
        'Release date': game['release_date'],
        'Estimated owners': game['estimated_owners'],
        'Peak CCU': game['peak_ccu'],
        'Required age': game['required_age'],
        'Price': game['price'],
        'DiscountDLC count': game['dlc_count'],
        'About the game': game['detailed_description'],
        'Supported languages': game['supported_languages'],
        'Full audio languages': game['full_audio_languages'],
        'Reviews': game['reviews'],
        'Header image': game['header_image'],
        'Website': game['website'],
        'Support url': game['support_url'],
        'Support email': game['support_email'],
        'Windows': game['windows'],
        'Mac': game['mac'],
        'Linux': game['linux'],
        'Metacritic score': game['metacritic_score'],
        'Metacritic url': game['metacritic_url'],
        'User score': game['user_score'],
        'Positive': game['positive'],
        'Negative': game['negative'],
        'Score rank': game['score_rank'],
        'Achievements': game['achievements'],
        'Recommendations': game['recommendations'],
        'Notes': game['notes'],
        'Average playtime forever': game['average_playtime_forever'],
        'Average playtime two weeks': game['average_playtime_2weeks'],
        'Median playtime forever': game['median_playtime_forever'],
        'Median playtime two weeks': game['median_playtime_2weeks'],
        'Developers': developers_str,
        'Publishers': publishers_str,
        'Categories': categories_str,
        'Genres': genres_str,
        'Tags': tags_str,
        'Screenshots': screenshots_str,
        'Movies': movies_str
    }
    games_data.append(game_dict)

# Create DataFrame
df = pd.DataFrame(games_data)

Do some preprocessing of the text in the description column and save it into the processed_description column

Preprocessing does:
- convert to lower case
- remove anything that is not a letter
- remove extra whitespace
- tokenize by splitting on whitespace
- remove stopwords belonging to nltk.corpus.stopwords.words('english')
- Lemmatize with WordNetLemmatizer()

In [3]:
df[ ['Categories', 'Genres', 'Tags', 'Developers', 'Publishers', 'Supported languages']]

,Categories,Genres,Tags,Developers,Publishers,Supported languages
0,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",Perpetual FX Creative,Perpetual FX Creative,[English]
1,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",Rusty Moyher,Wild Rooster,"[English, French, Italian, German, Spanish - S..."
2,Single-player,"Action,Adventure,Indie,Strategy",,Campião Games,Campião Games,"[English, Portuguese - Brazil]"
3,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",Odd Critter Games,Odd Critter Games,"[English, French, Italian, German, Spanish - S..."
4,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",Unusual Games,Unusual Games,"[English, Spanish - Spain]"
...,...,...,...,...,...,...
97405,"Single-player,Family Sharing",Casual,,Femdom Game World,Femdom Game World,"[English, Russian]"
97406,"Single-player,Steam Achievements,Partial Contr...","Casual,Simulation,Strategy",,Forever Entertainment S. A.,Forever Entertainment S. A.,"[English, French, Italian, German, Spanish - S..."
97407,Single-player,"Strategy,Free To Play",,Carlos Garrido,DigiPen Intsitute of Technology,[English]
97408,"Single-player,Steam Achievements,Family Sharing","Adventure,Casual,Indie",,CryneX,"CryneX,CryTechGames",[English]


In [4]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

def clean_text(text):
    """
    Clean text by removing special characters and extra whitespace
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

def process_text(text):
    """
    Process game description by removing stopwords, lemmatizing, and tokenizing
    """
    # Clean the text first
    text = clean_text(text)
    
    # Simple tokenization by splitting on whitespace
    tokens = text.split()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join back into a string
    processed_text = ' '.join(tokens)
    
    return processed_text

def process_list_field(text):
    """Process comma-separated fields while preserving the list structure"""
    # Handle arrays/lists
    if isinstance(text, (list, np.ndarray)):
        text = ','.join(map(str, text))
    
    # Handle NaN/empty
    if pd.isna(text) or str(text).strip() == '':
        return ''
        
    # Process each item in the comma-separated string
    items = str(text).split(',')
    processed_items = [process_text(item) for item in items]
    return ','.join(processed_items)

def process_game_data(df):
    """Process all relevant text fields in the dataframe"""
    processed_df = df.copy()
    
    # Process regular text fields
    text_fields = ['About the game', 'Name']
    for field in text_fields:
        processed_df[f'processed_{field.lower().replace(" ", "_")}'] = processed_df[field].apply(
            lambda x: process_text(str(x)) if pd.notnull(x) else ''
        )
    
    # Process list fields
    list_fields = ['Categories', 'Genres', 'Tags', 'Developers', 'Publishers', 'Supported languages']
    for field in list_fields:
        # Convert lists/arrays to strings first if needed
        if isinstance(processed_df[field].iloc[0], (list, np.ndarray)):
            processed_df[field] = processed_df[field].apply(lambda x: ','.join(map(str, x)) if isinstance(x, (list, np.ndarray)) else x)
        processed_df[f'processed_{field.lower()}'] = processed_df[field].apply(process_list_field)
    
    return processed_df

[nltk_data] Downloading package punkt to /home/ben/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ben/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ben/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
processed_df = process_game_data(df) # takes about a 5 minutes

In [10]:
processed_df.to_csv('processed_df.csv') # to avoid processing again

In [13]:
processed_df['processed_supported languages']

0                                                  english
1        english,french,italian,german,spanish spain,ja...
2                                english,portuguese brazil
3        english,french,italian,german,spanish spain,ja...
4                                    english,spanish spain
                               ...                        
97405                                      english,russian
97406    english,french,italian,german,spanish spain,si...
97407                                              english
97408                                              english
97409                                              english
Name: processed_supported languages, Length: 97410, dtype: object

Making the index with more fields than we care for takes about 7 and a half minutes. Most of the processing time here is from storing the description I'm guessing.

In [14]:
from whoosh.index import create_in
from whoosh.fields import *
import shutil
import os

# Define the schema with processed fields
schema = Schema(
    app_id=ID(stored=True),
    name=TEXT(stored=True),
    processed_name=TEXT(stored=True),
    processed_description=TEXT(stored=True),
    processed_categories=KEYWORD(stored=True, commas=True),
    processed_genres=KEYWORD(stored=True, commas=True),
    processed_tags=KEYWORD(stored=True, commas=True),
    processed_developers=KEYWORD(stored=True, commas=True),
    processed_publishers=KEYWORD(stored=True, commas=True),
    processed_languages=KEYWORD(stored=True, commas=True),
    price=NUMERIC(stored=True, numtype=float),
    release_date=TEXT(stored=True),
    metacritic_score=NUMERIC(stored=True)
)

# Create and populate index
if os.path.exists("index"):
    shutil.rmtree("index")
os.mkdir("index")

ix = create_in("index", schema)
writer = ix.writer()

print("adding docs to writer")
for _, game in processed_df.iterrows():
    writer.add_document(
        app_id=str(game['AppID']),
        name=game['Name'],
        processed_name=game['processed_name'],
        processed_description=game['processed_about_the_game'],
        processed_developers=game['processed_developers'],
        processed_publishers=game['processed_publishers'],
        processed_categories=game['processed_categories'],
        processed_genres=game['processed_genres'],
        processed_tags=game['processed_tags'],
        processed_languages=game['processed_supported languages'],
        price=float(game['Price']),
        release_date=game['Release date'],
        metacritic_score=int(game['Metacritic score'])
    )

writer.commit()

adding docs to writer


In [32]:
game['processed_categories']

'single player,steam achievement,full controller support,steam cloud,family sharing'

Example query using Whoosh. I think a good next step is modifying the MyIndexReader from assignment three to get postings or something along those lines from the processed descriptions for terms. Then also want description length and collection length for performing dirichlet smoothing. For the powerpoint, it cold also be interesting to make some basic plots about the distribution of word frequencies, distribution of categories/genres etc. 

In [46]:
# Example search usage
from whoosh.qparser import QueryParser
print("*"*10, "\nSearching on description")
with ix.searcher() as searcher:
    # Process the search query the same way as the indexed content
    search_term = 'fun'
    query = QueryParser("processed_description", schema).parse(search_term)
    results = searcher.search(query)
    for r in results:
        print(f"{r['name']} - {r['app_id']}")

print("*"*10, "\nSearching on categories")
with ix.searcher() as searcher:
    # Process the search query the same way as the indexed content
    query = QueryParser("processed_categories", schema).parse('"single player"')
    results = searcher.search(query)
    for r in results:
        print(f"{r['name']} - {r['app_id']}")

********** 
Searching on description
Ultimate Spinner Simulator - Unstress Yourself - 730030
Сrime Сity - 1937600
Water Girls - 1398720
Spooky Spins Returns : Crazy Cash Edition - Slots - 2671990
Halloween Trouble 2 - 1519800
Nature Escapes 2 - 2221630
My Little Car Wash - Cars & Trucks Roleplaying Game for Kids - 2633230
Build Lands - 1921390
NightFeed - 2718150
Tower Of God: One Wish - 1199010
********** 
Searching on categories
Galactic Bowling - 20200
Train Bandit - 655370
Jolt Project - 1732930
Henosis™ - 1355720
Two Weeks in Painland - 1139950
Wartune Reborn - 1469160
TD Worlds - 1659180
Legend of Rome - The Wrath of Mars - 1968760
MazM: Jekyll and Hyde - 1178150
Deadlings: Rotten Edition - 320150


note that the indexing of categories is not perfect, as you can see below, the categories are stored as full strings. so an exact match model can't accept 'cross platform multiplayer' because the parser will split that into cross AND platform AND multiplayer. To match these you need double quotes '"cross platform multiplayer"' to exactly match the full string.

In [20]:
# see unique categories before indexing by whoosh
# processed_df['Categories'].unique()

# And check if categories are being properly indexed
with ix.searcher() as searcher:
    # List all terms in the categories field
    print(list(searcher.lexicon("processed_categories")))

[b'app purchase', b'caption available', b'co op', b'commentary available', b'cross platform multiplayer', b'family sharing', b'full controller support', b'hdr available', b'includes level editor', b'includes source sdk', b'lan co op', b'lan pvp', b'mmo', b'mod', b'mod require hl', b'multi player', b'online co op', b'online pvp', b'partial controller support', b'pvp', b'remote play phone', b'remote play tablet', b'remote play together', b'remote play tv', b'shared split screen', b'shared split screen co op', b'shared split screen pvp', b'single player', b'stats', b'steam achievement', b'steam cloud', b'steam leaderboards', b'steam timeline', b'steam trading card', b'steam turn notification', b'steam workshop', b'steamvr collectible', b'tracked controller support', b'tracked motion controller support', b'valve anti cheat enabled', b'vr', b'vr support', b'vr supported']
